In [46]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np
import numbers
from FSSLibrary import FFTBasic as fft

import skyboxdatapy as skb

## Functions

In [47]:
def getDictFromDataframe(df, indexes, col_save_name = None):
    
    # Index 3 is Time
    df2 = df.iloc[:, [3] + indexes]

    inner = {}

    col_save_name = df2.columns if col_save_name is None else col_save_name

    for (col, name) in zip(df2.columns, col_save_name):
        arr = df2[col].to_numpy()        
        inner[name] = arr

    return inner

In [ ]:
def process_skybox_case(
        dfListRow,         
        fSampling,
        pathPattern="../data_nosync/*/Measure_XLS/",          
        calibTestName=None,
        calib_sheet_names = ["ConversionFactor", "ConversionOffset"]):
    
    """
    Process a Skybox test case by reading measurement and calibration data from Excel files.
        
    Parameters
    ----------
    dfListRow : pandas.Series
        A row from the test list excel sheet containing test case metadata
    pathPattern : str, optional
        Path pattern to Excel files directory 
        (default: "../data_nosync/*/Measure_XLS/")
    data_sheet_names : list of tuple, optional
        List of tuples containing (save_name, sheet_name) to read from measurement files 
        (default: [("MP3Filtered", "Filtered"), ("MP3RawValue", "RawValue")])
    defaultData : str, optional
        Name of the default dataset to use 
        (default: 'MP3RawValue')
    calibTestName : str, optional
        Name of the calibration test case to read 
        (default: None)
    calib_sheet_names : list of str, optional
        List of sheet names to read from calibration files 
        (default: ["ConversionFactor", "ConversionOffset"])
        
    Returns
    -------
    None
        Saves processed data to a .mat file in the Measure_MAT directory
    """

    print()
    
    sheets = {}

    # =====================================================================
    # Read Calibration Sheets
    if(calibTestName):
        
        calibFile = skb.io.find_unique_file(pathPattern, calibTestName, extension="xlsx")
        print(f"Processing Calibration file:\n{calibFile}")    
          
        for sheet_name in calib_sheet_names:

            df = pd.read_excel(calibFile, sheet_name=sheet_name, nrows=5)    
            df = df.drop(
                columns=['Index','Date-Time','Time','Counter','LED-chan100'], 
                errors='ignore'
            )    
            
            inner = {}
            for col in df.columns:
                arr = df[col][0] # Taking only first row
                if isinstance(arr, (np.integer, int)):
                    arr = np.float64(arr)
                inner[col] = arr
            sheets[sheet_name] = inner                


    # =====================================================================
    # Read Data Sheets
    caseName = dfListRow['C2']
    caseFile = skb.io.find_unique_file(pathPattern, caseName, extension="csv")
    print(f"Processing Case file:\n{caseFile}")

    df = pd.read_csv(caseFile, delimiter=',', low_memory=False)
    print(df.head())

    # Remove Index, Date and Counter
    index_base = [5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38, 41, 44, 47, 50, 53, 56, 59, 62, 65, 68]
    # index_base = list(range(5,23))
    dfTemp = df.iloc[1:10, [3] + index_base]
    col_save_name = dfTemp.columns

    # Get MP3Filtered
    indexes = index_base
    inner = getDictFromDataframe(df, indexes)
    sheets["MP3Filtered"] = inner.copy()
    inner.update({"reference": "MP3Filtered"})
    sheets["DefaultData"] = inner.copy()

    # Get MP3RawValue
    indexes = [i+1 for i in index_base]
    inner = getDictFromDataframe(df, indexes, col_save_name)
    sheets["MP3RawValue"] = inner.copy()

    # Get MP3Voltage
    indexes = [i+2 for i in index_base]
    inner = getDictFromDataframe(df, indexes, col_save_name)
    sheets["MP3Voltage"] = inner.copy()

    

    # =====================================================================
    # Wave properties
    sheets['TestProperties'] = {
        'airGapAtMPL': dfListRow['C12']/1000,  # meters
        'calibrationFile': calibTestName if calibTestName else 'None',
        'depthAtMPL': (dfListRow['C7']-33)/1000,    # meters
        'depthAtWM': dfListRow['C7']/1000,    # meters
        'focusingLocation': dfListRow['C11'],
        'fSampling': fSampling,              # Hz
        'repeatType': dfListRow['C4'],
        'testName': dfListRow['C2'],
        'testType': dfListRow['C3'],
        'useTest': dfListRow['C6'],
        'waveAmplitude': dfListRow['C9']/1000, # meters
        'wavePeriod': float(dfListRow['C10']) if isinstance(dfListRow['C10'], numbers.Number) else dfListRow['C10'],  # seconds
        'waveType': dfListRow['C8'],
        'remarks': dfListRow['C16']
    }


    # =====================================================================
    # Save to .mat file
    outFile = caseFile.replace('Measure_XLS', 'Measure_MAT').replace('.csv', '.mat')
    skb.io.save_hdf5_mat(outFile, sheets)



## Process one case

In [51]:
# Test case containing calibration data
calibTestName = "Test_d1021_Calib"

# Data test cases
fileList = glob.glob("../AllTest_log/Test_log_PLT.xlsx")
print(fileList)
dfList = pd.read_excel(fileList[0])    

print(dfList.columns)

index = 86
dfListRow = dfList.iloc[index]
print(f"\n=== Starting {dfListRow['C2']} ===")

process_skybox_case(
    dfListRow, 
    fSampling=2000.0,
    pathPattern="../data_nosync/*/Measure_XLS/",    
    calibTestName=calibTestName,
    calib_sheet_names = ["ConversionFactor", "ConversionOffset"]
)

print(f"=== Finished {dfListRow['C2']} ===\n")


['../AllTest_log/Test_log_PLT.xlsx']
Index(['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11',
       'C12', 'C13', 'C14', 'C15', 'C16'],
      dtype='object')

=== Starting Test430 ===

Processing Calibration file:
../data_nosync/d1021/Measure_XLS/Test_d1021_Calib.xlsx
Processing Case file:
../data_nosync/d1021/Measure_XLS/Test430_PLT775_d53_SSJ111_H146_T165_R00b.csv
   Unnamed: 0  Index          Date      Time  Counter       WG01     WG01.1  \
0           0      0  45936.423281  0.000000   200738 -64.897884 -64.889925   
1           1      1  45936.423281  0.060099   200738 -64.945639 -64.929721   
2           2      2  45936.423281  0.060599   200738 -64.953598 -64.929721   
3           3      3  45936.423281  0.061099   200738 -64.937680 -64.929721   
4           4      4  45936.423281  0.061599   200738 -64.945639 -64.929721   

    WG01.2       WG02     WG02.1  ...      L1.2        L2      L2.1      L2.2  \
0 -2.64027 -64.582643 -64.794363  ... -0.452390 -0.16025

## For debugging

In [50]:
stop

NameError: name 'stop' is not defined

In [ ]:
caseName = "../data_nosync/*/Measure_XLS/Test377"
# caseName = "../Membrane_Test/Measure_XLS/Mem008"
wvT = 1.0

files = glob.glob(caseName + "*.csv")
print(files)
caseFile = files[0]

fSampling = 2000 # Hz


In [ ]:
df = pd.read_csv(caseFile, delimiter=',', low_memory=False)
print(df.head())

In [ ]:
def getDictFromDataframe(df, indexes):
    
    # Index 3 is Time
    df2 = df.iloc[:, [3] + indexes]

    inner = {}
    for col in df2.columns:
        arr = df2[col].to_numpy()        
        inner[col] = arr

    return inner

In [ ]:
sheets = {}

# Remove Index, Date and Counter
index_base = [5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38, 41, 44, 47, 50, 53, 56, 59, 62, 65, 68]


# Get MP3Filtered
indexes = index_base
inner = getDictFromDataframe(df, indexes)
sheets["MP3Filtered"] = inner
inner.update({"reference": "MP3Filtered"})
sheets["DefaultData"] = inner

# Get MP3RawValue
indexes = [i+1 for i in index_base]
inner = getDictFromDataframe(df, indexes)
sheets["MP3RawValue"] = inner

# Get MP3Voltage
indexes = [i+2 for i in index_base]
inner = getDictFromDataframe(df, indexes)
sheets["MP3Voltage"] = inner

outFile = caseFile.replace('Measure_XLS', 'Measure_MAT').replace('.csv', '.mat')
skb.io.save_hdf5_mat(outFile, sheets)

In [ ]:
# # excel_path = files[0].replace('.csv', '.xlsx')
# # df2.to_excel(excel_path, index=False, sheet_name='RawValue')

# excel_path = files[0].replace('.csv', 'b.csv')
# df2.to_csv(excel_path)

In [ ]:
# caseName = "../d1008/Measure_XLS/Test341"

# wvT = 1.0

# files = glob.glob(caseName + "*.csv")
# print(files)

# fSampling = 2048 # Hz

# df = pd.read_csv(files[0])
# # Print first 5 rows to confirm structure
# print(df.head())